```
This software is a part of GPU Ocean.

Copyright (C) 2018, 2019 SINTEF Digital

This notebook sets up a demo for handling dry states (land).

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys
from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
#rc('figure', figsize=(16.0, 12.0))
#rc('animation', html='html5')
plt.rcParams["animation.html"] = "jshtml"

#Import our simulator
from SWESimulators import KP07, CDKLM16, PlotHelper, Common, WindStress, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
%cuda_context_handler gpu_ctx

In [ ]:
#Create output directory for images
#imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
#os.makedirs(imgdir)
#print("Saving images to " + imgdir)

In [ ]:
# Set initial conditions common to all simulators
sim_args = {
"gpu_ctx": gpu_ctx,
"nx": 10, "ny": 200,
#"dx": 200.0, "dy": 200.0,
"dx": 1.0, "dy": 1.0,
"dt": 0.1,
"g": 9.81,
"f": 0, # 0.0012,
"coriolis_beta": 0,
"r": 0.0
}

In [ ]:

def sim_animation(sim, T, substep=10.0, skips=0):
    
    if skips > 0:
        t = sim.step(skips*substep)
    
    const_x = int(sim.nx/2)
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    Hm = sim.downloadBathymetry()[1]
    simH_xsec = Hm[2:-2, const_x]

    
    #Create figure and plot initial conditions
    #fig = plt.figure(figsize=(12, 15))
    
    fig = plt.figure(figsize=(12,14))

    gs = fig.add_gridspec(5,3)
    ax_eta  = fig.add_subplot(gs[0:2, 0])
    ax_u    = fig.add_subplot(gs[0:2, 1])
    ax_v    = fig.add_subplot(gs[0:2, 2])
    ax_xsec = fig.add_subplot(gs[2, :])
    
    ax_xsec_hu = fig.add_subplot(gs[3, :])
    ax_xsec_hv = fig.add_subplot(gs[4, :])
    
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    #ax_eta = plt.subplot(1,3,1)
    #ax_eta = plt.subplot(2,3,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-5.5, vmax=5.5, extent=domain_extent)
    
    #ax_u = plt.subplot(1,3,2)
    #ax_u = plt.subplot(2,3,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='bottom', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    #ax_v = plt.subplot(1,3,3)
    #ax_v = plt.subplot(2,3,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='bottom', vmin=-1.5, vmax=1.5, extent=domain_extent)
    
    #ax_xsec = plt.subplot(2,3,4)
    sp_etaxsec, = ax_xsec.plot(range(sim.ny), eta1[:, const_x], color='xkcd:ocean')
    sp_zero,    = ax_xsec.plot(range(sim.ny), np.zeros_like(eta1[:,0]), color='xkcd:ocean', linestyle='dashed')
    sp_Hconst,  = ax_xsec.plot(range(sim.ny), -simH_xsec, color='xkcd:brown', zorder=10)
    ax_xsec.set_ylabel('eta at x = ' + str(const_x))
    ax_xsec.set_xlabel('y cell')
    ax_xsec.grid()
    
    sp_hvxsec, = ax_xsec_hv.plot(range(sim.ny), v1[:, const_x], color='xkcd:ocean')
    ax_xsec_hv.set_ylabel('hv at x = ' + str(const_x))
    ax_xsec_hv.set_xlabel('y_cell')
    ax_xsec_hv.set_ylim([-10, 10])
    ax_xsec_hv.grid()
    
    sp_huxsec, = ax_xsec_hu.plot(range(sim.ny), u1[:, const_x], color='xkcd:ocean')
    ax_xsec_hu.set_ylabel('hu at x = ' + str(const_x))
    ax_xsec_hu.set_xlabel('y_cell')
    ax_xsec_hu.set_ylim([-10, 10])
    ax_xsec_hu.grid()
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(substep)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(u1)
        
        fig.sca(ax_v)
        sp_v.set_data(v1)
        
        #fig.sca(ax_xsec)
        xsec = np.maximum(eta1[:, const_x], -simH_xsec)
        sp_etaxsec.set_ydata(xsec)
        
        sp_hvxsec.set_ydata(v1[:, const_x])
        sp_huxsec.set_ydata(u1[:, const_x])
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print("."+str(np.sum(eta1))+".", end='')
        #print(".", end='')
        

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    
    return anim

In [ ]:
def linearCase(low, high, use_cdklm=True, T=60, substep=100, bump=False):
    ghosts = np.array([2,2,2,2]) # north, east, south, west
    dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
                 sim_args["nx"] + ghosts[1]+ghosts[3])

    #H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
    H = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32)
    eta0 = np.zeros(dataShape, dtype=np.float32)
    hu0 = np.zeros(dataShape, dtype=np.float32)
    hv0 = np.zeros(dataShape, dtype=np.float32)

    # Create sloping bathymetry
    
    linearBathymetryY(H, sim_args['nx'], sim_args['ny'], sim_args['dx'], sim_args['dy'], ghosts, high, low)
    
    #H -= 5
    #h_mid_cell = int((sim_args["ny"]+2)/2)
    #H[h_mid_cell,:] -= 2
    
    
    if bump:
        H2 = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32)
        addBump(H2, sim_args['nx'], sim_args['ny'], sim_args['dx'], sim_args['dy'], 0.5, 0.75, 0.5*sim_args['dx'], ghosts)
        H2 *= -5
        H += H2
        
    
    plt.imshow(H, origin='lower', interpolation='none')
    plt.colorbar()
    print('H.shape: ', H.shape)

    #sdjgass
    #Create dam break in to south of domain
    addStraightDam(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts, 3)

    #fig = plt.figure()
    #plt.imshow(eta0, origin='lower', interpolation='none')
    #plt.colorbar()

    #H[-3, :] = H[-4,:] + (H[-4,:] - H[-5,:])
    #H[-2, :] = H[-3,:] + (H[-3,:] - H[-4,:])
    #H[-1, :] = H[-2,:] + (H[-2,:] - H[-3,:])
    
    #H = np.vstack((H[1:,:], np.flip(H, axis=0)))[::2,:]
    
    #Initialize simulator
    fvm_args = {"H": H, "eta0": eta0, "hu0": hu0, "hv0": hv0,
                "boundary_conditions": Common.BoundaryConditions(1,2,1,2)}

    sim = None
    if use_cdklm:
        sim = CDKLM16.CDKLM16(**fvm_args, **sim_args, rk_order=1)
    else:
        sim = KP07.KP07(**fvm_args, **sim_args, use_rk2=False)

    #sim_H = sim.downloadBathymetry()
    #print('sim_H.shape: ', sim_H[0].shape)
    #fig = plt.figure()
    #plt.imshow(sim_H[0], origin='lower', interpolation='none')
    #plt.colorbar()


    return sim
    #Run a simulation and plot it
    #return sim_animation(sim, T=T, substep=substep)

In [ ]:
#sim = linearCase(0.5, 10, use_cdklm=False)
#sim_animation(sim, T=100, substep=189) #, skips=50)

In [ ]:
reload(CDKLM16)
sim = linearCase(-4, 6, use_cdklm=True)
sim.dt=0.01
#sim_animation(sim, T=30, substep=0.01, skips=1685)#1860)
#sim_animation(sim, T=100, substep=0.5)

#sim_animation(sim, T=8, substep=sim.dt)
#sim_animation(sim, T=80, substep=sim.dt) #, skips=200)
sim_animation(sim, T=100, substep=0.5)


In [ ]:
print(sim.dy)
trastrseseers

In [ ]:
cdklm = linearCase(-4, 10, use_cdklm=True)
kp = linearCase(-4, 10, use_cdklm=False)

hi_cdklm = cdklm.downloadBathymetry()[0]
hm_cdklm = cdklm.downloadBathymetry()[1]
print("hi_cdklm.shape", hi_cdklm.shape)
print("hm_cdklm.shape", hm_cdklm.shape)
hi_kp = kp.downloadBathymetry()[0]
hm_kp = kp.downloadBathymetry()[1]
print(hi_kp.shape)
fig = plt.figure()
plt.imshow(hi_cdklm)

fig = plt.figure()
plt.plot(np.arange(-2,4), hi_cdklm[:6,4], label='hi')
plt.plot(np.arange(6) - 1.5,hm_cdklm[:6,4], label='hm')
plt.grid()
plt.legend()

fig = plt.figure()
plt.plot(np.arange(-2,4), hi_cdklm[-6:,4], label='hi')
plt.plot(np.arange(6) - 2.5,hm_cdklm[-6:,4], label='hm')
plt.grid()
plt.legend()

fig = plt.figure(figsize=(15,4))
ax = fig.add_subplot(1, 1, 1)
minor_ticks = np.arange(-2,23)
ax.plot(minor_ticks, -hi_cdklm[:,4], label='hi')
ax.plot(np.arange(24) - 1.5, -hm_cdklm[:,4])
ax.legend()
ax.set_xticks(minor_ticks, minor=True)
ax.grid(which='minor')

fig = plt.figure()
plt.plot(np.arange(-2,4), hi_kp[:6,4], label='hi')
plt.plot(np.arange(6) - 1.5,hm_kp[:6,4], label='hm')
plt.grid()
plt.legend()

fig = plt.figure()
plt.plot(np.arange(-2,4), hi_kp[-6:,4], label='hi')
plt.plot(np.arange(6) - 2.5,hm_kp[-6:,4], label='hm')
plt.grid()
plt.legend()

In [ ]:
reload(CDKLM16)
#sim = linearCase(-1, 10, use_cdklm=False)
#sim_animation(sim, T=10, substep=189, skips=50)
sim = linearCase(0.5, 10, use_cdklm=True, bump=True)

sim.dt = 0.01
#sim_animation(sim, T=30, substep=0.01, skips=1955)#1860)
sim_animation(sim, T=100, substep=0.5)
#sim_animation(sim, T=8, substep=sim.dt)


In [ ]:
print(sim.dt, sim.dx, sim.local_size)

sdhgsdsdasdgasdg

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

# Create sloping bathymetry
linearBathymetryY(H, sim_args['nx'], sim_args['ny'], sim_args['dx'], sim_args['dy'], ghosts, 10, -5)
#plt.imshow(H, origin='lower', interpolation='none')
#plt.colorbar()
#print('H.shape: ', H.shape)

#Create dam break in to south of domain
addStraightDam(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts, 3)

#fig = plt.figure()
#plt.imshow(eta0, origin='lower', interpolation='none')
#plt.colorbar()


#Initialize simulator
ctcs_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "use_rk2": True,
            "boundary_conditions": Common.BoundaryConditions(1,2,1,2)}
sim = KP07.KP07(**ctcs_args, **sim_args)

#sim_H = sim.downloadBathymetry()
#print('sim_H.shape: ', sim_H[0].shape)
#fig = plt.figure()
#plt.imshow(sim_H[0], origin='lower', interpolation='none')
#plt.colorbar()



#Run a simulation and plot it
sim_animation(sim, T=60, substep=100)

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

#Create bump in to lower left of domain for testing
addCentralBump(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts)

#Initialize simulator
ctcs_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "rk_order": 2}
sim = CDKLM16.CDKLM16(**ctcs_args, **sim_args)

#Run a simulation and plot it
sim_animation(sim, T=50)

In [ ]:
str(cuda.Context.get_cache_config())